In [ ]:
# ===========================
# Script 2: Train with QLoRA
# ===========================
!pip install transformers peft accelerate bitsandbytes datasets -q

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
import torch

# --------------------------
# LOAD MODEL
# --------------------------
model_name = "codellama/CodeLlama-7b-hf"  # or try starcoder
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# --------------------------
# LOAD DATASET
# --------------------------
dataset = load_dataset("json", data_files="synthetic_c_bugs.json")

# Create input text with classification label + explanation
def format_example(example):
    bug_label = "BUG" if example["label"]==1 else "NO_BUG"
    return f"Prompt: {example['prompt']}\nLabel: {bug_label}\nExplanation: {example['response']}"

def tokenize_function(example):
    text = format_example(example)
    return tokenizer(text, truncation=True, padding="max_length", max_length=512)

tokenized = dataset.map(tokenize_function, batched=False)

# --------------------------
# LORA CONFIG
# --------------------------
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# --------------------------
# TRAINING CONFIG
# --------------------------
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=1,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
)
trainer.train()

# --------------------------
# SAVE MODEL
# --------------------------
model.save_pretrained("c_bug_lora")
print("✅ Model saved as c_bug_lora")

# --------------------------
# TESTING PIPELINE
# --------------------------
from transformers import pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

test_prompt = """Find bug in this C code:

#include <stdio.h>
#include <stdlib.h>
int main() {
  int *p = malloc(5 * sizeof(int));
  for(int i=0;i<=5;i++) p[i]=i; // potential bug?
  return 0;
}"""

print(pipe(test_prompt, max_length=300, do_sample=True)[0]["generated_text"])
